# Project Notebook: Optimizing DataFrames and Processing in Chunks

## 1. Introduction 

In this project, we'll practice working with chunked dataframes and optimizing a dataframe's memory usage. We'll be working with financial lending data from Lending Club, a marketplace for personal loans that matches borrowers with investors. You can read more about the marketplace on its website.

The Lending Club's website lists approved loans. Qualified investors can view the borrower's credit score, the purpose of the loan, and other details in the loan applications. Once a lender is ready to back a loan, it selects the amount of money it wants to fund. When the loan amount the borrower requested is fully funded, the borrower receives the money, minus the origination fee that Lending Club charges.

We'll be working with a dataset of loans approved from 2007-2011 (https://bit.ly/3H2XVgC). We've already removed the desc column for you to make our system run more quickly.

If we read in the entire data set, it will consume about 67 megabytes of memory. Let's imagine that we only have 10 megabytes of memory available throughout this project, so you can practice the concepts you learned in the last two lessons.

**Tasks**

1. Read in the first five lines from `loans_2007.csv` (https://bit.ly/3H2XVgC) and look for any data quality issues.

2. Read in the first 1000 rows from the data set, and calculate the total memory usage for these rows. Increase or decrease the number of rows to converge on a memory usage under five megabytes (to stay on the conservative side).

In [110]:
# Importing pandas
import pandas as pd
pd.options.display.max_columns = 99

# Your code goes here
loans = pd.read_csv("loans_2007.csv", nrows=1000)
print(loans.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          1000 non-null   int64  
 1   member_id                   1000 non-null   float64
 2   loan_amnt                   1000 non-null   float64
 3   funded_amnt                 1000 non-null   float64
 4   funded_amnt_inv             1000 non-null   float64
 5   term                        1000 non-null   object 
 6   int_rate                    1000 non-null   object 
 7   installment                 1000 non-null   float64
 8   grade                       1000 non-null   object 
 9   sub_grade                   1000 non-null   object 
 10  emp_title                   949 non-null    object 
 11  emp_length                  983 non-null    object 
 12  home_ownership              1000 non-null   object 
 13  annual_inc                  1000 n

In [3]:
loans

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [4]:
print(loans)

        id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  1077501  1296599.0     5000.0       5000.0           4975.0   36 months   
1  1077430  1314167.0     2500.0       2500.0           2500.0   60 months   
2  1077175  1313524.0     2400.0       2400.0           2400.0   36 months   
3  1076863  1277178.0    10000.0      10000.0          10000.0   36 months   
4  1075358  1311748.0     3000.0       3000.0           3000.0   60 months   

  int_rate  installment grade sub_grade                 emp_title emp_length  \
0   10.65%       162.87     B        B2                       NaN  10+ years   
1   15.27%        59.83     C        C4                     Ryder   < 1 year   
2   15.96%        84.33     C        C5                       NaN  10+ years   
3   13.49%       339.31     C        C1       AIR RESOURCES BOARD  10+ years   
4   12.69%        67.79     B        B5  University Medical Group     1 year   

  home_ownership  annual_inc verification_status  

In [5]:
#removing empty cells
loans = pd.read_csv("loans_2007.csv", nrows=1000)
New_loans = loans.dropna()
print(New_loans)

          id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
1    1077430  1314167.0     2500.0       2500.0      2500.000000   60 months   
3    1076863  1277178.0    10000.0      10000.0     10000.000000   36 months   
4    1075358  1311748.0     3000.0       3000.0      3000.000000   60 months   
5    1075269  1311441.0     5000.0       5000.0      5000.000000   36 months   
6    1069639  1304742.0     7000.0       7000.0      7000.000000   60 months   
..       ...        ...        ...          ...              ...         ...   
995  1057629  1289394.0     2425.0       2425.0      2425.000000   36 months   
996  1057621  1289385.0     6950.0       6950.0      6950.000000   36 months   
997  1057787  1289153.0    12375.0      12375.0     12344.464785   36 months   
998  1057770  1289135.0    35000.0      35000.0     33906.194198   60 months   
999  1057275  1288835.0    14000.0      14000.0     14000.000000   60 months   

    int_rate  installment grade sub_gra

In [6]:
# Read in the first 1000 rows from the data set, and calculate the total memory usage for these rows. 
#Increase or decrease the number of rows to converge on a memory usage under five megabytes (to stay on the conservative side).

In [7]:
New_loans = pd.read_csv("loans_2007.csv", nrows=1000)
New_loans.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          1000 non-null   int64  
 1   member_id                   1000 non-null   float64
 2   loan_amnt                   1000 non-null   float64
 3   funded_amnt                 1000 non-null   float64
 4   funded_amnt_inv             1000 non-null   float64
 5   term                        1000 non-null   object 
 6   int_rate                    1000 non-null   object 
 7   installment                 1000 non-null   float64
 8   grade                       1000 non-null   object 
 9   sub_grade                   1000 non-null   object 
 10  emp_title                   949 non-null    object 
 11  emp_length                  983 non-null    object 
 12  home_ownership              1000 non-null   object 
 13  annual_inc                  1000 n

## 2. Exploring the Data in Chunks

Let's familiarize ourselves with the columns to see which ones we can optimize. In the first lesson, we explored column types by reading in the full dataframe. In this project, let's try to understand the column types better while using dataframe chunks.

**Tasks**

For each chunk:
* How many columns have a numeric type? 
* How many have a string type?
* How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?
* Which float columns have no missing values and could be candidates for conversion to the integer type?
* Calculate the total memory usage across all of the chunks.

In [112]:
#How many columns have a numeric type?
chunker = pd.read_csv("loans_2007.csv", chunksize=1000, header=0)
numeric = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'complex']

for chunk in chunker:
  chunker_numeric = chunk.select_dtypes(include=numeric)
  print(len(chunker_numeric.columns))




31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
30
31
31
30


In [113]:
# How many have a string type?
chunker = pd.read_csv("loans_2007.csv", chunksize=1000, header=0)
for chunk in chunker:
  strings = chunk.select_dtypes(include=['object']).columns
  print(len(strings))

21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
22
21
21
22


In [ ]:
#How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?
#Which float columns have no missing values and could be candidates for conversion to the integer type?
#Calculate the total memory usage across all of the chunks

In [10]:
chunker = pd.read_csv("loans_2007.csv", chunksize=1000, header=0)
for unique in chunker:
  for col in unique.select_dtypes(include=['object']):
    print(f"{col}: {len(unique[col].unique())}")
   

term: 2
int_rate: 34
grade: 7
sub_grade: 34
emp_title: 920
emp_length: 12
home_ownership: 3
verification_status: 3
issue_d: 1
loan_status: 4
pymnt_plan: 1
purpose: 13
title: 544
zip_code: 378
addr_state: 42
earliest_cr_line: 268
revol_util: 566
initial_list_status: 1
last_pymnt_d: 54
last_credit_pull_d: 52
application_type: 1
term: 2
int_rate: 35
grade: 7
sub_grade: 35
emp_title: 907
emp_length: 12
home_ownership: 3
verification_status: 3
issue_d: 2
loan_status: 6
pymnt_plan: 1
purpose: 13
title: 538
zip_code: 382
addr_state: 43
earliest_cr_line: 287
revol_util: 590
initial_list_status: 1
last_pymnt_d: 53
last_credit_pull_d: 53
application_type: 1
term: 2
int_rate: 35
grade: 7
sub_grade: 34
emp_title: 917
emp_length: 12
home_ownership: 3
verification_status: 3
issue_d: 2
loan_status: 5
pymnt_plan: 1
purpose: 13
title: 553
zip_code: 384
addr_state: 43
earliest_cr_line: 297
revol_util: 567
initial_list_status: 1
last_pymnt_d: 55
last_credit_pull_d: 55
application_type: 1
term: 2
int_rate

In [11]:
chunker = pd.read_csv("loans_2007.csv", chunksize=1000, header=0)
for unique in chunker:
  for col in unique.select_dtypes(include=['object']):
    sum = len(unique[col])
    unique_sum = len(unique[col].unique())
    percentage = (unique_sum/sum)*100
    if percentage < 50:
      print(f"{col}: {percentage}%")

term: 0.2%
int_rate: 3.4000000000000004%
grade: 0.7000000000000001%
sub_grade: 3.4000000000000004%
emp_length: 1.2%
home_ownership: 0.3%
verification_status: 0.3%
issue_d: 0.1%
loan_status: 0.4%
pymnt_plan: 0.1%
purpose: 1.3%
zip_code: 37.8%
addr_state: 4.2%
earliest_cr_line: 26.8%
initial_list_status: 0.1%
last_pymnt_d: 5.4%
last_credit_pull_d: 5.2%
application_type: 0.1%
term: 0.2%
int_rate: 3.5000000000000004%
grade: 0.7000000000000001%
sub_grade: 3.5000000000000004%
emp_length: 1.2%
home_ownership: 0.3%
verification_status: 0.3%
issue_d: 0.2%
loan_status: 0.6%
pymnt_plan: 0.1%
purpose: 1.3%
zip_code: 38.2%
addr_state: 4.3%
earliest_cr_line: 28.7%
initial_list_status: 0.1%
last_pymnt_d: 5.3%
last_credit_pull_d: 5.3%
application_type: 0.1%
term: 0.2%
int_rate: 3.5000000000000004%
grade: 0.7000000000000001%
sub_grade: 3.4000000000000004%
emp_length: 1.2%
home_ownership: 0.3%
verification_status: 0.3%
issue_d: 0.2%
loan_status: 0.5%
pymnt_plan: 0.1%
purpose: 1.3%
zip_code: 38.4%
addr_s

In [12]:
chunker = pd.read_csv("loans_2007.csv", chunksize=1000, header=0)
for chunk in chunker:
  strings = chunk.select_dtypes(include=['float16', 'float32', 'float64'])
  print(strings.isnull().sum())

member_id                     0
loan_amnt                     0
funded_amnt                   0
funded_amnt_inv               0
installment                   0
annual_inc                    0
dti                           0
delinq_2yrs                   0
inq_last_6mths                0
open_acc                      0
pub_rec                       0
revol_bal                     0
total_acc                     0
out_prncp                     0
out_prncp_inv                 0
total_pymnt                   0
total_pymnt_inv               0
total_rec_prncp               0
total_rec_int                 0
total_rec_late_fee            0
recoveries                    0
collection_recovery_fee       0
last_pymnt_amnt               0
collections_12_mths_ex_med    0
policy_code                   0
acc_now_delinq                0
chargeoff_within_12_mths      0
delinq_amnt                   0
pub_rec_bankruptcies          0
tax_liens                     0
dtype: int64
member_id                  

In [13]:
chunker = pd.read_csv("loans_2007.csv", chunksize=1000, header=0)
for chunk in chunker:
  print(chunk.info(memory_usage="deep"))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          1000 non-null   int64  
 1   member_id                   1000 non-null   float64
 2   loan_amnt                   1000 non-null   float64
 3   funded_amnt                 1000 non-null   float64
 4   funded_amnt_inv             1000 non-null   float64
 5   term                        1000 non-null   object 
 6   int_rate                    1000 non-null   object 
 7   installment                 1000 non-null   float64
 8   grade                       1000 non-null   object 
 9   sub_grade                   1000 non-null   object 
 10  emp_title                   949 non-null    object 
 11  emp_length                  983 non-null    object 
 12  home_ownership              1000 non-null   object 
 13  annual_inc                  1000 n

## 3. Optimizing String Columns

We can achieve the greatest memory improvements by converting the string columns to a numeric type. Let's convert all of the columns where the values are less than 50% unique to the category type, and the columns that contain numeric values to the `float` type.

While working with dataframe chunks:
* Determine which string columns you can convert to a numeric type if you clean them. For example, the `int_rate` column is only a string because of the % sign at the end.
* Determine which columns have a few unique values and convert them to the category type. For example, you may want to convert the grade and `sub_grade` columns.
Based on your conclusions, perform the necessary type changes across all chunks. * Calculate the total memory footprint, and compare it with the previous one.

In [14]:
# Your code goes here
chunker = pd.read_csv("loans_2007.csv", chunksize=1000, header=0)
for unique in chunker:
  for col in unique.select_dtypes(include=['object']):
    sum = len(unique[col])
    unique_sum = len(unique[col].unique())
    percentage = (unique_sum/sum)*100
    if percentage < 50:
      unique[col] = unique[col].astype('category')
      

In [36]:
chunker = pd.read_csv("loans_2007.csv", chunksize=1000, header=0)
for chunk in chunker:
  chunk['term'] = chunk['term'].str.replace(' months', '')
  chunk['int_rate'] = chunk['int_rate'].str.replace('%', '')
  chunk['zip_code'] = chunk['zip_code'].str.replace('xx', '')
  chunk['revol_util'] = chunk['revol_util'].str.replace('%', '')

  cols = ['term', 'int_rate', 'zip_code', 'revol_util']
  for col in cols:
    chunk[col] = chunk[col].astype('float')
print(chunk)


                                                    id  member_id  loan_amnt  \
42000                                           247286   247257.0     6000.0   
42001                                           246996   244258.0    17250.0   
42002                                           246720   246706.0    13000.0   
42003                                           246535   246427.0    12000.0   
42004                                           246197   217842.0     4000.0   
...                                                ...        ...        ...   
42533                                            72176    70868.0     2525.0   
42534                                            71623    70735.0     6500.0   
42535                                            70686    70681.0     5000.0   
42536  Total amount funded in policy code 1: 471701350        NaN        NaN   
42537          Total amount funded in policy code 2: 0        NaN        NaN   

       funded_amnt  funded_amnt_inv  te

## 4. Optimizing Numeric Columns

It looks like we were able to realize some powerful memory savings by converting to the category type and converting string columns to numeric ones.

Now let's optimize the numeric columns using the `pandas.to_numeric()` function.

**Tasks**

While working with dataframe chunks:
* Identify float columns that contain missing values, and that we can convert to a more space efficient subtype.
* Identify float columns that don't contain any missing values, and that we can convert to the integer type because they represent whole numbers.
* Based on your conclusions, perform the necessary type changes across all chunks.
* Calculate the total memory footprint and compare it with the previous one.




In [105]:

chunker = pd.read_csv("loans_2007.csv", chunksize=1000, header=0)
for cols in chunker:
  float_cols = cols.select_dtypes(include=['float16', 'float32', 'float64'])
  num_cols = float_cols.columns[float_cols.isna().any()].tolist()
  cols[num_cols] = cols[num_cols].fillna(0)
  mem = (cols.memory_usage(deep=True).sum())*0.000001
print(cols.info(memory_usage="deep"))

  
  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538 entries, 42000 to 42537
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          538 non-null    object 
 1   member_id                   538 non-null    float64
 2   loan_amnt                   538 non-null    float64
 3   funded_amnt                 538 non-null    float64
 4   funded_amnt_inv             538 non-null    float64
 5   term                        536 non-null    object 
 6   int_rate                    536 non-null    object 
 7   installment                 538 non-null    float64
 8   grade                       536 non-null    object 
 9   sub_grade                   536 non-null    object 
 10  emp_title                   499 non-null    object 
 11  emp_length                  536 non-null    object 
 12  home_ownership              536 non-null    object 
 13  annual_inc                  5

After replacing the NaN values with zeros, the memory has remained the same.


## Next Steps

We've practiced optimizing a dataframe's memory footprint and working with dataframe chunks. Here's an idea for some next steps:

Create a function that automates as much of the work you just did as possible, so that you could use it on other Lending Club data sets. This function should:

* Determine the optimal chunk size based on the memory constraints you provide.

* Determine which string columns can be converted to numeric ones by removing the `%` character.

* Determine which numeric columns can be converted to more space efficient representations.


In [107]:
# Your code goes here
chunker = pd.read_csv("loans_2007.csv", chunksize=1000, header=0)
for unique in chunker:
  for col in unique.select_dtypes(include=['object']):
    sum = len(unique[col])
    unique_sum = len(unique[col].unique())
    percentage = (unique_sum/sum)*100
    if percentage < 50:
      unique[col] = unique[col].astype('category')
for cols in chunker:
  float_cols = cols.select_dtypes(include=['float16', 'float32', 'float64'])
  num_cols = float_cols.columns[float_cols.isna().any()].tolist()
  cols[num_cols] = cols[num_cols].fillna(0)
  mem = (cols.memory_usage(deep=True).sum())*0.000001